In [44]:
##########################################################################################
# Import library
import sys
import os
import json
import numpy as np
import pandas as pd
from pandas.io.json import json_normalize
import matplotlib.pyplot as plt
import seaborn as sns
color = sns.color_palette()

from plotly import tools
import plotly.offline as py
py.init_notebook_mode(connected=True)
import plotly.graph_objs as go

from sklearn import model_selection, preprocessing, metrics
import lightgbm as lgb
sys.path.append('.')
sys.path.append('./code/feature')
from kit import *
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD
from timeit import time
pd.set_option('display.max_columns', 500)

In [55]:
train_df = pd.read_csv("./data/train.csv",dtype={'fullVisitorId': 'str'})
test_df = pd.read_csv("./data/test.csv",dtype={'fullVisitorId': 'str'})
print(train_df.shape)
print(test_df.shape)
print(train_df.columns)

(50000, 39)
(50000, 39)
Index(['channelGrouping', 'date', 'fullVisitorId', 'visitId', 'visitNumber',
       'visitStartTime', 'device_browser', 'device_deviceCategory',
       'device_isMobile', 'device_operatingSystem', 'geoNetwork_city',
       'geoNetwork_continent', 'geoNetwork_country', 'geoNetwork_metro',
       'geoNetwork_networkDomain', 'geoNetwork_region',
       'geoNetwork_subContinent', 'totals_bounces', 'totals_hits',
       'totals_newVisits', 'totals_pageviews', 'totals_sessionQualityDim',
       'totals_timeOnSite', 'totals_totalTransactionRevenue',
       'totals_transactionRevenue', 'totals_transactions',
       'trafficSource_adContent',
       'trafficSource_adwordsClickInfo.adNetworkType',
       'trafficSource_adwordsClickInfo.gclId',
       'trafficSource_adwordsClickInfo.isVideoAd',
       'trafficSource_adwordsClickInfo.page',
       'trafficSource_adwordsClickInfo.slot', 'trafficSource_campaign',
       'trafficSource_isTrueDirect', 'trafficSource_keyword',
 

In [56]:
# Hits columns are filtered out

In [57]:
cat_feat = [
    'channelGrouping',
    'device_browser',
    'device_deviceCategory',
    'device_isMobile',
    'device_operatingSystem',
    'geoNetwork_city',
    'geoNetwork_continent',
    'geoNetwork_country',
    'geoNetwork_metro',
    'geoNetwork_networkDomain',
    'geoNetwork_region',
    'geoNetwork_subContinent',
    'totals_bounces',
    'totals_newVisits',
    'trafficSource_adwordsClickInfo.adNetworkType',
    'trafficSource_adwordsClickInfo.isVideoAd',
    'trafficSource_adwordsClickInfo.page',
    'trafficSource_adwordsClickInfo.slot',
    'trafficSource_campaign',
    'trafficSource_isTrueDirect',
    'trafficSource_medium',
    'customDimensions_value'
]

more_action_feat = [
    'date',
    'fullVisitorId',
    'visitId',
    'visitStartTime',
    'trafficSource_adContent',
    'trafficSource_adwordsClickInfo.gclId',
    'trafficSource_keyword',
    'trafficSource_referralPath',
    'trafficSource_source',
]
num_feat = [
    'visitNumber',
    'totals_hits',
    'totals_pageviews',
    'totals_sessionQualityDim',
    'totals_timeOnSite',
    'totals_transactionRevenue',
    'totals_transactions',
]


In [58]:
# 1. understanding the difference between
f1 = 'totals_transactionRevenue'
f2 = 'totals_totalTransactionRevenue'
f3 = 'fullVisitorId'
for df in test_df,train_df:
#     print(f1)
    df[f1].fillna(0,inplace=True)
    df[f1] = df[f1].astype(float)
    df[f2].fillna(0,inplace=True)
    df[f2] = df[f2].astype(float)


In [61]:
ff1 = train_df[train_df[f1]!=0][[f1,f2,f3]]
ff2 = test_df[test_df[f1]!=0][[f1,f2,f3]]
ff2

,totals_transactionRevenue,totals_totalTransactionRevenue,fullVisitorId
268,2.199000e+07,2.499000e+07,0567632072087918290
353,1.499900e+08,1.519900e+08,0034941760139500427
369,7.730000e+06,9.730000e+06,7417113168654151999
381,7.662000e+07,8.462000e+07,5963345090934142475
391,4.398000e+07,4.598000e+07,6525673967695077450
397,1.050000e+07,1.450000e+07,9228745493176485536
410,4.047000e+07,4.647000e+07,957476142738569837
417,4.637000e+07,5.537000e+07,3549058200878432496
419,1.899000e+07,1.999000e+07,6064513912864833263
447,7.831000e+07,8.531000e+07,7946376510906023468


In [49]:
train_df[train_df[f3] == "1447279966112382026"][[f1,f2,f3]]

,totals_transactionRevenue,totals_totalTransactionRevenue,fullVisitorId
678,56090000.0,63090000.0,1447279966112382026
1935,0.0,0.0,1447279966112382026


In [50]:
train_df[[f1,f2]].corr(method='pearson')

,totals_transactionRevenue,totals_totalTransactionRevenue
totals_transactionRevenue,1.000000,0.964261
totals_totalTransactionRevenue,0.964261,1.000000


In [63]:
train_df['date']

0        20171016
1        20171016
2        20171016
3        20171016
4        20171016
5        20171016
6        20171016
7        20171016
8        20171016
9        20171016
10       20171016
11       20171016
12       20171016
13       20171016
14       20171016
15       20171016
16       20171016
17       20171016
18       20171016
19       20171016
20       20171016
21       20171016
22       20171016
23       20171016
24       20171016
25       20171016
26       20171016
27       20171016
28       20171016
29       20171016
           ...   
49970    20180103
49971    20180103
49972    20180103
49973    20180103
49974    20180103
49975    20180103
49976    20180103
49977    20180103
49978    20180103
49979    20180103
49980    20180103
49981    20180103
49982    20180103
49983    20180103
49984    20180103
49985    20180103
49986    20180103
49987    20180103
49988    20180103
49989    20180103
49990    20180103
49991    20180103
49992    20180103
49993    20180103
49994    2

In [64]:
test_df['date']

0        20180511
1        20180511
2        20180511
3        20180511
4        20180511
5        20180511
6        20180511
7        20180511
8        20180511
9        20180511
10       20180511
11       20180511
12       20180511
13       20180511
14       20180511
15       20180511
16       20180511
17       20180511
18       20180511
19       20180511
20       20180511
21       20180511
22       20180511
23       20180511
24       20180511
25       20180511
26       20180511
27       20180511
28       20180511
29       20180511
           ...   
49970    20181005
49971    20181005
49972    20181005
49973    20181005
49974    20181005
49975    20181005
49976    20181005
49977    20181005
49978    20181005
49979    20181005
49980    20181005
49981    20181005
49982    20181005
49983    20181005
49984    20181005
49985    20181005
49986    20181005
49987    20181005
49988    20181005
49989    20181005
49990    20181005
49991    20181005
49992    20181005
49993    20181005
49994    2